## Word Embedding
---


In [78]:
# Import
from nltk.tokenize import word_tokenize
from gensim.models.phrases import Phrases, Phraser
import fuzzysearch
import fuzzywuzzy
import pandas as pd
import PyPDF2

pdfFileObj = open('20221118-FULL.pdf', 'rb') #Create pdf object
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 

In [2]:
allText = ""
for i in range(100): #Iterating over n pages 
    pageObj = pdfReader.getPage(i)
    page = pageObj.extractText()
    allText += page #Store page content into string variable

allText;

#### Split Text into persons

---

In [3]:
# Preprocessing
allText = allText.replace(';','')

alltext_string_all = allText.replace('Legal basis',';Legal basis').replace('Identity information',';Identity information').replace('Programme', ';Programme').replace('Birth information',';Birth information').replace('Citizenship information',';Citizenship information').replace('Contact information',';Contact information').replace('Remark',';Remark')
testListSplit = alltext_string_all.split('reference number: EU')
alltext_string_all[:1000]


'EUROPEAN COMMISSION\nFinancial Stability, Financial Services and Capital Markets Union (DG FISMA)\nEuropean Union\nConsolidated Financial Sanctions List\nThis list has been updated on 18/11/2022 11:37European Union Consolidated Financial Sanctions List\nTable of contents\n1. Introduction3 \n2. Individuals or persons3 \n3. Entities or groups631 \n4. Disclaimer792 \nPage 2 on 792European Union Consolidated Financial Sanctions List\n1. INTRODUCTION\nThe present document contains the Consolidated List of persons, groups and entities subject, under EU Sanctions, to an asset freeze and the prohibition to\nmake funds and economic resources available to them. The latest version of this file is here.\n2. INDIVIDUALS OR PERSONS\nEU reference number: EU.1787.1\n;Legal basis: 2017/404 (OJ L63)\n;Programme: AFG - Afghanistan\n;Identity information:\n• Name/Alias: Sirajuddin Jallaloudine Haqqani  Function: Na’ib Amir (Deputy Commander)  \n• Name/Alias: Khalifa  \n• Name/Alias: Saraj Haqani  \n• Nam

In [4]:
# Create dataframe
dbn_df = pd.DataFrame([sub.split(";") for sub in testListSplit])
dbn_df.fillna("",inplace=True)
dbn_df.columns = ['EU Reference', 'Legal basis', 'Programme', 'Identity information', 'Birth information', 'Citizenship information', 'Contact information', 'Remark','','','','','','','','',]



In [7]:
dbn_df

,EU Reference,Legal basis,Programme,Identity information,Birth information,Citizenship information,Contact information,Remark,,,,,,,,
0,"EUROPEAN COMMISSION\nFinancial Stability, Fina...",,,,,,,,,,,,,,,
1,.1787.1\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,Identity information:\n• Name/Alias: Sirajuddi...,Birth information:\n• Birth date: Circa from 1...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Mi...",Remark: Heading the Haqqani Network as of late...,,,,,,,,
2,.2272.60\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,Identity information:\n• Name/Alias: Nasiruddi...,Birth information:\n• Birth date: Circa from 1...,"Remark: Neka District, Paktika Province, Afgha...",Citizenship information:\n• Citizenship: Afgha...,Contact information: \n• Address: Pakistan \n,Remark: A leader of the Haqqani Network (TAe.0...,,,,,,,
3,.2274.87\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,Identity information:\n• Name/Alias: Gul Agha ...,Birth information:\n• Birth date: Circa 1972 ...,Contact information: \n• Address: Pakistan \n...,Remark: Belongs to Ishaqzai tribe. INTERPOL-UN...,,,,,,,,,
4,.2291.0\n,Legal basis: 2017/404 (OJ L63)\n,Programme: AFG - Afghanistan\n,Identity information:\n• Name/Alias: Amir Abdu...,Birth information:\n• Birth date: Circa 1972 ...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Ka...",Remark: Believed to be in Afghanistan/Pakistan...,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,.3631.49\n,Legal basis: 2017/396 (OJ L60)\n,Programme: COD - Democractic Republic of Congo\n,Identity information:\n• Name/Alias: Mathieu C...,Birth information:\n• Birth date: 08/10/1970 ...,"Remark: Place of birth: Bunia, Ituri\nProvince...",Citizenship information:\n• Citizenship: Congo...,"Contact information: \n• Address: Congo, Democ...",Remark: Mathieu Chui Ngudjolo was the Chief of...,,,,,,,
384,.3632.14\n,Legal basis: 2017/396 (OJ L60)\n,Programme: COD - Democractic Republic of Congo...,Identity information:\n• Name/Alias: Floribert...,Birth information:\n• Birth date: 23/05/1971 ...,Citizenship information:\n• Citizenship: Congo...,Remark: President of FNI\nEU,,,,,,,,,
385,.3633.76\n,Legal basis: 2017/396 (OJ L60)\n,Programme: COD - Democractic Republic of Congo\n,Identity information:\n• Name/Alias: 'Chairman...,Birth information:\n• Birth date: 06/02/1967 ...,Citizenship information:\n• Citizenship: Congo...,,,,,,,,,,
386,.3634.41\n,Legal basis: 2017/396 (OJ L60)\n,Programme: COD - Democractic Republic of Congo\n,Identity information:\n• Name/Alias: Fred Irak...,"Birth information:\n• Birth place: Rwanda, Kin...",Citizenship information:\n• Citizenship: Rwanda\n,"Contact information: \n• Address: Congo, Democ...",Remark: As of June 2016\nEU,,,,,,,,


## Data Cleanup 
---

In [20]:
def cleanupDataframe(dataframe):
    dataframe['Legal basis'] = dataframe['Legal basis'].apply(lambda x: x.replace('Legal basis: ','').replace('\n',''))
    dataframe['Programme'] = dataframe['Programme'].apply(lambda x: x.replace('Programme: ','').replace('\n',''))
    dataframe['Identity information'] = dataframe['Identity information'].apply(lambda x: x.replace('Identity information:','').replace('\n','').replace('Name/Alias:',';').replace('•','').replace('Function:',''))
    dataframe.replace('^\s+', '', regex=True, inplace=True) #front
    dataframe.replace('\s+$', '', regex=True, inplace=True) #end
    
    return dataframe

def stripDataframe(dataframe):
    dataframe['Identity information'] = dataframe['Identity information'].apply(lambda x: x.strip())
    
    return dataframe

def listCreation(dataframe):
    identityInformation = []
    for i in dataframe:
        temp = dataframe.tolist()
        identityInformation.append(temp)
    return identityInformation


In [23]:
cleanupDataframe(dbn_df)
stripDataframe(dbn_df)


,EU Reference,Legal basis,Programme,Identity information,Birth information,Citizenship information,Contact information,Remark,,,,,,,,
0,"EUROPEAN COMMISSION\nFinancial Stability, Fina...",,,,,,,,,,,,,,,
1,.1787.1,2017/404 (OJ L63),AFG - Afghanistan,; Sirajuddin Jallaloudine Haqqani Na’ib Amir...,Birth information:\n• Birth date: Circa from 1...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Mi...",Remark: Heading the Haqqani Network as of late...,,,,,,,,
2,.2272.60,2017/404 (OJ L63),AFG - Afghanistan,; Nasiruddin Haqqani A leader of the Haqqani...,Birth information:\n• Birth date: Circa from 1...,"Remark: Neka District, Paktika Province, Afgha...",Citizenship information:\n• Citizenship: Afgha...,Contact information: \n• Address: Pakistan,Remark: A leader of the Haqqani Network (TAe.0...,,,,,,,
3,.2274.87,2017/404 (OJ L63),AFG - Afghanistan,; Gul Agha Ishakzai ; Mullah Gul Agha ; Mu...,Birth information:\n• Birth date: Circa 1972 ...,Contact information: \n• Address: Pakistan \n...,Remark: Belongs to Ishaqzai tribe. INTERPOL-UN...,,,,,,,,,
4,.2291.0,2017/404 (OJ L63),AFG - Afghanistan,; Amir Abdullah Former Kandahar Province Dep...,Birth information:\n• Birth date: Circa 1972 ...,Citizenship information:\n• Citizenship: Afgha...,"Contact information: \n• Address: Pakistan, Ka...",Remark: Believed to be in Afghanistan/Pakistan...,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,.3631.49,2017/396 (OJ L60),COD - Democractic Republic of Congo,; Mathieu Chui Ngudjolo Chief of Staff and f...,Birth information:\n• Birth date: 08/10/1970 ...,"Remark: Place of birth: Bunia, Ituri\nProvince...",Citizenship information:\n• Citizenship: Congo...,"Contact information: \n• Address: Congo, Democ...",Remark: Mathieu Chui Ngudjolo was the Chief of...,,,,,,,
384,.3632.14,2017/396 (OJ L60),COD - Democractic Republic of CongoPage 99 on ...,; Floribert Ngabu Njabu ; Floribert Njabu ...,Birth information:\n• Birth date: 23/05/1971 ...,Citizenship information:\n• Citizenship: Congo...,Remark: President of FNI\nEU,,,,,,,,,
385,.3633.76,2017/396 (OJ L60),COD - Democractic Republic of Congo,; 'Chairman' ; 'Papa Six' ; ģenerālis Nkun...,Birth information:\n• Birth date: 06/02/1967 ...,Citizenship information:\n• Citizenship: Congo...,,,,,,,,,,
386,.3634.41,2017/396 (OJ L60),COD - Democractic Republic of Congo,; Fred Irakeza FDLR-FOCA Colonel ; Félicie...,"Birth information:\n• Birth place: Rwanda, Kin...",Citizenship information:\n• Citizenship: Rwanda,"Contact information: \n• Address: Congo, Democ...",Remark: As of June 2016\nEU,,,,,,,,


## Fuzzy string matching
---

In [ ]:
from thefuzz import fuzz, process 

# Preprocessing
n = dbn_df['Identity information'].to_string().split()
n

In [93]:
name = 'Pascal'
#x = input('gib dein scheiß hier ein')

# Search function
def findBusinessPartner(input, name, threshold):
    count = 0
    for i in input:
        count += 1
        if fuzz.ratio(i, name) >= threshold:
            print(fuzz.ratio(i, name), name)
    
        elif count == len(input): 
             print(f'Your requested Business partner {name} with fuzzy threshold {threshold} cannot be found on the EU Sanctions List')


In [96]:
name = 'Pascal'
findBusinessPartner(n, name, 40)

43 Pascal
40 Pascal
55 Pascal
50 Pascal
40 Pascal
40 Pascal
43 Pascal
62 Pascal
40 Pascal
40 Pascal
40 Pascal
43 Pascal
40 Pascal
40 Pascal
40 Pascal
40 Pascal
50 Pascal
46 Pascal
55 Pascal
46 Pascal
40 Pascal
40 Pascal
40 Pascal
40 Pascal
40 Pascal
40 Pascal
40 Pascal
46 Pascal
40 Pascal
40 Pascal
43 Pascal
55 Pascal
55 Pascal
40 Pascal
46 Pascal
43 Pascal
40 Pascal
40 Pascal
40 Pascal
40 Pascal
46 Pascal
55 Pascal
55 Pascal
43 Pascal
43 Pascal
43 Pascal
55 Pascal
40 Pascal
43 Pascal
55 Pascal
Your requested Business partner Pascal with fuzzy threshold 40 cannot be found on the EU Sanctions List


## Word2Vec
---

In [79]:
import gensim
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from nltk.tokenize import sent_tokenize, word_tokenize
from matplotlib import pyplot
 
# #Preprocessing
# data = dbn_df['Identity information'] = dbn_df['Identity information'].apply(lambda content: [x for x in content.split(';')]) #Translating 'Identity information' into list object

# sentences_tokenized = []
# for i in data:
#     print(i)
#     # s = i.lower
#     # myPunc = '!"#%&$()*+-./:;,<=>?@[\\]^_`{|}~'
#     # s = s.translate(s.maketrans(myPunc, ' '*len(myPunc)))
#     # s = s.translate(s.maketrans(string.digits, ' '*len(string.digits)))
#     # s = re.sub('\s+',' ',s)
#     # print(s)
#     # for sentence in s:    
#     #     tokens = sentence.split(' ')
#     #     sentences_tokenized.append(tokens)




In [77]:
# data1 = []
# for i in dbn_df['Identity information']:
#     # temp = []
#     # for j in i:
#     #     temp.append(j.lower())
#     # data.append(temp)
#     data1.append(i.lower())


ImportError: cannot import name 'word2vec' from 'gensim' (c:\Users\DELSTEI9\AppData\Local\Programs\Python\Python310\lib\site-packages\gensim\__init__.py)

In [80]:
# Creating Word2Vec
model = Word2Vec(
    sentences = n,
    #size = 50,
    window = 10,
    #iter = 20,
    
)




In [81]:
model.wv.key_to_index #Ausgabe aller Key-Vektor Paare in Word2Vec model


{'a': 0,
 '.': 1,
 'i': 2,
 'l': 3,
 'e': 4,
 ';': 5,
 'r': 6,
 'A': 7,
 'h': 8,
 'n': 9,
 't': 10,
 'd': 11,
 'o': 12,
 'u': 13,
 'm': 14,
 'M': 15,
 'v': 16,
 'j': 17,
 'S': 18,
 's': 19,
 'T': 20,
 '2': 21,
 '1': 22,
 'I': 23,
 'а': 24,
 '3': 25,
 'K': 26,
 'k': 27,
 'V': 28,
 'H': 29,
 'N': 30,
 'е': 31,
 'b': 32,
 'J': 33,
 'А': 34,
 'и': 35,
 'O': 36,
 ':': 37,
 'R': 38,
 'c': 39,
 'р': 40,
 'л': 41,
 'н': 42,
 'в': 43,
 'U': 44,
 'L': 45,
 'E': 46,
 'g': 47,
 'D': 48,
 'f': 49,
 '4': 50,
 '7': 51,
 '6': 52,
 '5': 53,
 'y': 54,
 '8': 55,
 '0': 56,
 '9': 57,
 'о': 58,
 'z': 59,
 'B': 60,
 'G': 61,
 'ч': 62,
 'Y': 63,
 'В': 64,
 'q': 65,
 'C': 66,
 'P': 67,
 'і': 68,
 'д': 69,
 'к': 70,
 'К': 71,
 'с': 72,
 'О': 73,
 'т': 74,
 'й': 75,
 'Н': 76,
 'И': 77,
 'F': 78,
 'С': 79,
 'я': 80,
 'М': 81,
 'p': 82,
 'ь': 83,
 'Z': 84,
 'м': 85,
 'Р': 86,
 '(': 87,
 'г': 88,
 'Е': 89,
 'x': 90,
 'У': 91,
 ')': 92,
 'Л': 93,
 'w': 94,
 'Д': 95,
 'Q': 96,
 'Т': 97,
 'Г': 98,
 ',': 99,
 'Б': 100,

In [82]:
ergebnisse = model.wv.most_similar(name, topn=5)

for y in ergebnisse:
    print(y)

KeyError: "Key 'Sirajuddin' not present in vocabulary"